In [ ]:
import dwave.system
import numpy as np
import matplotlib.pyplot as plt
import src.particle_funcs.distance_matrix as distance_matrix
import src.leap_funcs.qubo.q_matrix as q_matrix

In [ ]:
num_particles = 7

use_dimod_exact_solver = False

part_coords_n = np.zeros((num_particles,2))
part_coords_nm1 = np.zeros((num_particles,2))

for i in range(np.shape(part_coords_n)[0]):
    part_coords_n[i,:] = [0, i]
    part_coords_nm1[i,:] = [0.5*1, i]

fig_initial, axs_initial = plt.subplots(1,1)
axs_initial.scatter(part_coords_n[:,0], part_coords_n[:,1], label="n")
axs_initial.scatter(part_coords_nm1[:,0], part_coords_nm1[:,1], label="n-1")
fig_initial.legend()
fig_initial.show()

correct_sol = np.zeros(num_particles*num_particles)
for i in range(1, num_particles+1):
    correct_sol[(i-1)*num_particles + i -1] = 1.

print(correct_sol)

In [ ]:
distances = distance_matrix.calc_phi_ij(part_coords_n, part_coords_nm1)
print(distances)
print(np.max(distances))

In [ ]:
Q_dist = q_matrix.q_dist(distances)
assert (Q_dist.transpose() == Q_dist).all(), 'Q_dist is not symmetric'
with np.printoptions(precision=3):
    print(Q_dist)
max_abs = np.max(Q_dist)
#Q_dist /= max_abs
#with np.printoptions(precision=3):
#    print(Q_dist)

In [ ]:
Q_part = q_matrix.q_part(np.shape(distances)[0])
print(Q_part)

In [ ]:
Q_pos = q_matrix.q_pos(np.shape(distances)[0])
print(Q_pos)
print(Q_pos[0,:])

In [ ]:
Q_array = Q_dist + Q_part + Q_pos


with np.printoptions(precision=3, suppress=True):
    print(Q_array)


In [ ]:
Q_dict = q_matrix.Q_convert_to_dict(Q_array)
#Q_dict = q_matrix.Q_convert_to_dict(Q_array_rem)
print(Q_dict)

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite

with open('../API_Token.txt') as file:
    token = file.readline().rstrip()
    architecture = file.readline().rstrip()

#sampler = DWaveSampler(token = token, architecture='pegasus')
sampler = DWaveSampler(solver=dict(topology__type='pegasus'), token = token)

print(DWaveSampler._get_solver(sampler))

sampler = EmbeddingComposite(sampler)
sample = sampler.sample_qubo(Q_dict, num_reads = 10)

In [ ]:
print(sample)

In [ ]:
#usually is sorted for energies automatically
ids_sorted_energies = np.argsort(sample.data_vectors['energy'])

print(sample.record[ids_sorted_energies[0:7]])

In [ ]:
from src.leap_funcs.qubo import filter_samples

ids_valid_solutions= filter_samples.indices_of_valid_solutions(sample)
print(ids_valid_solutions)
print(sample.record[ids_valid_solutions])

In [ ]:
for sol in sample.record:
    if np.array_equal(correct_sol, sol[0]):
        print(sol)
for sol in sample.record[ids_valid_solutions]:
    if np.array_equal(correct_sol, sol[0]):
        print(sol)

In [ ]:
#print(np.sort(i_mats))
#print(np.sort(j_mats))
#print(np.intersect1d(i_mats, j_mats))
#tmp = sample.record[ids_sorted_energies[0]][0]
#tmp = np.insert(tmp, 6, 0)
#tmp = np.insert(tmp, 42, 0)
#tmp = np.insert(tmp, 48, 0)
#print(tmp)

print(sample.record[ids_sorted_energies[0]][0])
print(correct_sol)
print(np.array_equal(correct_sol, sample.record[ids_sorted_energies[0]][0]))
correct_sol - sample.record[ids_sorted_energies[0]][0]

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite

with open('../API_Token.txt') as file:
    token = file.readline().rstrip()
    architecture = file.readline().rstrip()

#sampler = DWaveSampler(token = token, architecture='pegasus')
sampler = DWaveSampler(solver=dict(topology__type='pegasus'), token = token)

print(DWaveSampler._get_solver(sampler))

sampler = EmbeddingComposite(sampler)
print(sampler)
#response = sampler.sample_qubo(Q_dict, num_reads=3)

In [ ]:
import dwave.inspector
print(dir(dwave.inspector))

In [ ]:
dwave.inspector.from_objects()

In [ ]:
dwave.inspector.show(sampler)

In [ ]:
response

In [ ]:
with open('../API_Token.txt') as file:
    token = file.readline().rstrip()
from dwave.system import DWaveSampler
import dwave.inspector
# Get solver
sampler = DWaveSampler(token=token, solver=dict(topology__type='pegasus'))
# Define a problem (actual qubits depend on the selected QPU's working graph)
h = {}
J = {(2136, 4181): -1, (2136, 2151): -0.5, (2151, 4196): 0.5, (4181, 4196): 1}
all(edge in sampler.edgelist for edge in J)     
# Sample
response = sampler.sample_ising(h, J, num_reads=10)    


In [ ]:
print(sampler)   

In [ ]:
sampler.properties

In [ ]:
import dwave.inspector

In [ ]:
dwave.inspector.show(response)